# 11 Session Persistence (Experiment)
Read first: `examples/14_session_persistence.py`

Save and resume a conversation. Run once to create state, run again to resume. State is stored under `~/.amplifier/demo_sessions`.


## Select a provider
Set `PROVIDER_KEY` to `sonnet`, `opus`, `gpt`, or `gpt-codex` and run the next cell.
The helper will set env vars (prompting if missing) and print the selection.


In [ ]:
# Choose your provider key (sonnet | opus | gpt | gpt-codex)
PROVIDER_KEY = "sonnet"

In [ ]:
# Resolve foundation/provider paths and ensure env var
import importlib.util
import os
from pathlib import Path


def find_repo_root() -> Path:
    candidates = [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]
    for p in candidates:
        if (p / "utils" / "providers.py").exists():
            return p
    return Path.cwd()


ROOT = find_repo_root()
providers_path = ROOT / "utils" / "providers.py"
spec = importlib.util.spec_from_file_location("providers_helper", providers_path)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)
select_provider = module.select_provider
print_provider_menu = module.print_provider_menu

FOUNDATION_PATH, PROVIDER_PATH, REQUIRED_ENV = select_provider(PROVIDER_KEY, foundation="minimal")
print_provider_menu(PROVIDER_KEY)
print(f"Selected: {PROVIDER_KEY} -> {PROVIDER_PATH.name}")
env_set = "Yes" if os.environ.get(REQUIRED_ENV) else "No"
print(f"Required env: {REQUIRED_ENV} (set? {env_set})")

In [ ]:
# Optional: set API key here
import getpass

if not os.getenv(REQUIRED_ENV):
    os.environ[REQUIRED_ENV] = getpass.getpass(f"Enter {REQUIRED_ENV}: ")
    print(f"Set {REQUIRED_ENV} for this kernel session.")
else:
    print(f"{REQUIRED_ENV} already set.")

In [ ]:
# Parameters (edit me)
SESSION_ID = "demo-workflow"

In [ ]:
import json
from datetime import datetime

from amplifier_foundation import load_bundle


class SimpleSessionPersistence:
    def __init__(self, storage_path: Path):
        self.storage_path = storage_path
        self.messages = []
        self.metadata = {}

    def save(self):
        state = {"metadata": self.metadata, "messages": self.messages, "saved_at": datetime.now().isoformat()}
        self.storage_path.parent.mkdir(parents=True, exist_ok=True)
        with open(self.storage_path, "w") as f:
            json.dump(state, f, indent=2)

    def load(self) -> bool:
        if not self.storage_path.exists():
            return False
        with open(self.storage_path) as f:
            state = json.load(f)
        self.metadata = state.get("metadata", {})
        self.messages = state.get("messages", [])
        return True

    def add_message(self, role: str, content: str):
        self.messages.append({"role": role, "content": content, "timestamp": datetime.now().isoformat()})

    def summary(self) -> str:
        lines = ["Previous conversation:"]
        for msg in self.messages[-5:]:
            lines.append("[{}] {}...".format(msg.get("role"), msg.get("content", "")[:80]))
        return "\n".join(lines)


async def run_once():
    storage_dir = Path.home() / ".amplifier" / "demo_sessions"
    storage_dir.mkdir(parents=True, exist_ok=True)
    state_file = storage_dir / f"{SESSION_ID}.json"
    persistence = SimpleSessionPersistence(state_file)
    resuming = persistence.load()

    foundation = await load_bundle(str(FOUNDATION_PATH))
    provider = await load_bundle(str(PROVIDER_PATH))
    composed = foundation.compose(provider)
    prepared = await composed.prepare()
    session = await prepared.create_session()

    async with session:
        if resuming:
            print("Resuming from", state_file)
            print(persistence.summary())
            prompt = input("Continue conversation (or type quit): ").strip()
            if prompt.lower() == "quit":
                persistence.save()
                return
            persistence.add_message("user", prompt)
            response = await session.execute(prompt)
            print("Response:")
            print(response)
            persistence.add_message("assistant", response)
        else:
            print("Starting new session; state will be saved to", state_file)
            prompt = "Explain asyncio.gather() in 2-3 sentences."
            persistence.add_message("user", prompt)
            response = await session.execute(prompt)
            print("Response:")
            print(response)
            persistence.add_message("assistant", response)

    persistence.save()
    print("Saved state to", state_file)


await run_once()

### Try this
- Run once to create state, then rerun to resume with the saved context.
- Change `SESSION_ID` to maintain multiple saved conversations.
- Edit the default prompt or add more turns before saving.
